## Data Cleaning and Processing

In [ ]:
# TODO 
https://medium.com/@farukdemirok/hybrid-recommender-system-811ee411b697 - proceesubf dups

In [47]:
import numpy as np
import pandas as pd

import ast

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer



In [ ]:
# # colsolidate into one dataframe
# df = four_star.append(five_star_clothing, ignore_index=True)
# df = df.append(five_star_not_clothing, ignore_index=True)
# df.shape

# # remove any rows with headers
# df = df.drop(df[df['pattern_id']=='pattern_id'].index)
# # drop douplicates
# df = df.drop_duplicates()

In [84]:
df = pd.read_csv('data/consolidated_patterns.csv', low_memory=False)

def clean_out_bad_data(df):
    # extra indicies from .csv appending and accidental douplicates
    df = df.drop(df[df['pattern_id']=='pattern_id'].index)
    df = df.drop_duplicates()
    
    # not enough projects (43451 rows)
    df = df.drop(df[df['projects_count'] <= 10].index)
    
    # drop rows with too many nulls (15795 rows)
    df = df.drop(df[df.isnull().sum(axis=1) >3].index)
    
    # drop if no category
    df = df.drop(df[df['categories'].isna()].index)

    return df

# Need to save and reload clean data!! 
df = clean_out_bad_data(df)
df.reset_index(inplace=True, drop=True)

df.to_csv('data/patterns_cleaned.csv', index=False)

# # INCLUDE WEIGHTED AVERAGE SCORE RATHER THAN RATING?

In [49]:
df.columns

Index(['pattern_id', 'name', 'name_permalink', 'favorites_count',
       'projects_count', 'difficulty_average', 'difficulty_count',
       'rating_average', 'queued_projects_count', 'rating_count',
       'pattern_type_names', 'pattern_type_clothing', 'photos_url',
       'pattern_needle_sizes', 'pattern_attributes', 'yardage_max', 'yardage',
       'generally_available', 'gauge', 'gauge_divisor', 'free', 'downloadable',
       'categories', 'yarn_weight_description'],
      dtype='object')

#### Categorical

In [8]:
# cat_columns = ['pattern_type_clothing','downloadable',
#                'free','categories', 'pattern_type_names','pattern_attributes','yarn_weight_description']
# # THERE IS A PATTERN TYPE ID _ JUST USE THAT?? instead of names??
# # MAYBE REGROUP AND ONE HOT ENCODE? OR READJUST AND LABEL ENCODE IDS?

In [1]:
# df[df.isnull().sum(axis=1) >3]

In [111]:
# # PATTERN_TYPES (ID AND NAMES)
# print(pat.pattern_type_id.unique())
# print(pat['pattern_type_names'].unique())
# len(pat.pattern_type_names.unique())
# # one hot encode?

[10  7 17  2  3  1  9  6  8 20 16  4 19 15  5 11 21 13 18]
['shawl' 'cardigan' 'child' 'socks' 'hat' 'scarf' 'other' 'pet' 'gloves'
 'shrug' 'toys' 'pullover' 'jacket' 'bag' 'baby' 'blanket' 'home' 'vest'
 'tee']


19

In [51]:
df.info()

# find percentage null

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132843 entries, 0 to 188156
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   pattern_id               132843 non-null  int64  
 1   name                     132843 non-null  object 
 2   name_permalink           132843 non-null  object 
 3   favorites_count          132843 non-null  int64  
 4   projects_count           132843 non-null  int64  
 5   difficulty_average       132843 non-null  float64
 6   difficulty_count         131547 non-null  float64
 7   rating_average           132843 non-null  float64
 8   queued_projects_count    132843 non-null  int64  
 9   rating_count             132841 non-null  float64
 10  pattern_type_names       132842 non-null  object 
 11  pattern_type_clothing    132842 non-null  object 
 12  photos_url               132843 non-null  object 
 13  pattern_needle_sizes     132843 non-null  object 
 14  patt

In [25]:
# columns to one_hot encode, or true, false?
# There are 19 pattern_type_names - not sure which one to use...
cols_to_one_hot = ['pattern_type_clothing','downloadable','free','pattern_type_names'] # note pattern_type_names 19
cols_to_count_vectorize = ['categories', 'pattern_attributes']
cols_to_transform = ['yarn_weight_description']

#### categories

In [58]:
df['categories']
def parse_out_single_category(df):
    df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x))
    df['categories'] = df['categories'].apply(lambda x: x[0])
    return df
df_categories = parse_out_single_category(df)

In [60]:
df_categories.categories
df_categories_dummies = pd.Series(df_categories['categories']).str.get_dummies()

In [62]:
df_categories_dummies.shape

(132843, 189)

#### Attributes

### Functions for DataFrame Function Transformer

In [19]:
def drop_identifying_columns(df):
    df.drop(columns = ['pattern_id','name', 
                        'name_permalink','photos_url', 'pattern_attributes', 
                       'pattern_type_names','categories','rating_count','favorites_count', 
                       'projects_count','rating_average',
                       'difficulty_count','queued_projects_count'], inplace= True)
    return df
#----------------------------------pattern attributes actually need to be vectorized
def consolidate_gauge(df):
    """ takes in gauge columns and normalizes them all to stiches per inch """
    try:
        df['gauge_per_inch'] = df.loc[:,'gauge']/df.loc[:,'gauge_divisor']
        df.drop(columns = ['gauge', 'gauge_divisor'], axis = 1, inplace = True)
    except:
        print("Error occured when consolidating gauge")
    return df

def encode_yarn_weights(df):
    """ uses yarn_weight_description to convert weights correspnding actual relative thicknesses
    indstry standards splits into 7 categories, but this goes more ganually """
    yarn_weights = {'Lace' : 1,
                    'Thread':1,
                    'Cobweb':1,
                    'Light Fingering':1.5,
                    'Fingering (14 wpi)': 2,
                    'Sport (12 wpi)': 3,
                    'DK / Sport' : 4,
                    'DK (11 wpi)' : 5,
                    'Worsted (9 wpi)':6,
                    'Aran / Worsted': 7,
                    'Aran (8 wpi)': 8,
                    'Bulky (7 wpi)':9,
                    'Super Bulky (5-6 wpi)':10,
                    'Jumbo (0-4 wpi)':11,
                    'No weight specified':5,
                    'Any gauge - designed for any gauge':5}
    ##--------------need to get yarn weight for other 
    try:
        df.replace({'yarn_weight_description':yarn_weights}, inplace=True)
        df.rename(columns = {'yarn_weight_description':'yarn_weight_code'}, inplace=True)
    except:
        print("okay - check out yarn_weight_description, something went wrong with the encoding")
    return df

# # NOT USED - using average for now
# def fill_max_yardage(df):
#     """ if there the yardage column is filled, but max_yardage is empty, it's because there isn't a 
#     range of yardage (in the case of one-sized non-clothing items).  This function fills that column in the c
#     case we only want to use min or max yardage """
#     df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
#     return df

def use_avg_yardage(df):
    df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
    df['yardage_avg'] = (df['yardage'] + df['yardage_max'])/2
    df.drop(columns=['yardage_max','yardage'], inplace=True)
    return df

def split_date_released(df):
    """ takes in 'generally_available' column and splits out into month and year 
    (to take into account seasonal purchases, and yearly impacts)"""
    try:
        df['generally_available'] = pd.to_datetime(df['generally_available'], utc=True)
        df['month_avail'] = df.generally_available.dt.month
        df['year_avail'] = df.generally_available.dt.year
     
        df.drop(columns = ['generally_available'], axis = 1, inplace = True)
    except:
        "print - Error occured when trying to split dates"
    return df

def get_list_metric_needles(x):
    """Get the list of metric needle sizes for knitting (can have several needle sizes for various parts of the pattern 
    and crochet hooks for finishing.  Generally the largest needle is the 
    one used for the main body (or largest portion of the project) """
    n=[]
    for i in range(len(x)):
        if x[i]['knitting'] == True:
            n.append(x[i]['metric'])
    return n

def get_needle_size(df):
    try:
        df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
    except:
        pass
    
    df['pattern_needle_sizes'] = df['pattern_needle_sizes'].apply(lambda x: get_list_metric_needles(x))
    df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: max(x) if (len(x)>=1) else None)
    return df

# def calculate_popularity(df):
#     pass
# # get_weighted_rating

def parse_out_single_category(df):
    df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x))
    df['categories'] = df['categories'].apply(lambda x: x[0])
    return df

def vectorize_categories(df):
    pass
# count_vect = CountVectorizer()

def vectorize_pattern_attributes(df):
    corpus = df['attributes'].tolist()
    
def impute_missing_values_or_zeros(df):
    df['difficulty_average'].fillna(df['difficulty_average'].median(), inplace=True)
    df['pattern_type_clothing'].fillna(df['pattern_type_clothing'].median(), inplace=True)
    df['pattern_needle_sizes'].fillna(df['pattern_needle_sizes'].median(), inplace=True)
    df['free'].fillna(df['free'].median(), inplace=True)
    df['downloadable'].fillna(df['downloadable'].median(), inplace=True)
    df['yarn_weight_code'].fillna(df['yarn_weight_code'].median(), inplace=True)
    df['gauge_per_inch'].fillna(df['gauge_per_inch'].median(), inplace=True)
#        'month_avail', 'year_avail', 
    df['yardage_avg'].fillna(df['yardage_avg'].median(), inplace=True)
    return df


# def one_hot_encode_binary(df):
    
#     feature_to_encode = ['pattern_type_clothing','free','downloadable'] 
#     dummies = pd.get_dummies(df[[feature_to_encode]], drop_first=True)
#     res = pd.concat([df, dummies], axis=1)
#     res = res.drop([feature_to_encode], axis=1)
#     return(res)

def apply_weighting_values(df):
    pass

#     using count isnt meaningful here ----------
# def get_weighted_difficulty_average(df):
#     overal_difficulty_average = df.difficulty_average.mean()

# def convert_to_sentence(x):
#     # concatenate words with '-'
    
#     # covert from list to sentence
#     x = ' '.join(word for word in x)
#     x = x.replace('-','')
#     #consolidatae and remove a number of words! right now 245-----------------------------
#     return x

# df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: ast.literal_eval(x))
# df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: convert_to_sentence(x))
# corpus = df['pattern_attributes']
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(corpus)
# features = vectorizer.get_feature_names()


#--------------------------------------------
# This one works lovely 
def transform_features(data):
    data['yardage_avg'] = use_avg_yardage(df)
    encode_yarn_weights(df)
    data['Total_Income_Log'] = np.log(data['ApplicantIncome'] + data['CoapplicantIncome'])
    data['LoanAmt_Term_Ratio_Log']=  np.log(data['LoanAmount']/data['Loan_Amount_Term'])
    data['LoanAmount_Log'] = np.log(data['LoanAmount'])
    data.drop(labels=['ApplicantIncome','CoapplicantIncome', 'LoanAmount'], axis=1, inplace=True)
    data.reset_index(inplace=True)
    return data

# add_total_income_log_object = FunctionTransformer(replace_income_with_total_income_log)
# add_loanamt_term_ratio_log_object = FunctionTransformer(add_LoanAmt_Term_Ratio_Log)
# add_loanamount_log_object = FunctionTransformer(replace_loanamount_with_loanamount_log)

transform_features = FunctionTransformer(transform_features)

Try one ginormous function

In [ ]:
def drop_identifying_columns(df):
    df.drop(columns = ['pattern_id','name', 
                        'name_permalink','photos_url', 'pattern_attributes', 
                       'pattern_type_names','categories','rating_count','favorites_count', 
                       'projects_count','rating_average',
                       'difficulty_count','queued_projects_count'], inplace= True)
    return df
#----------------------------------pattern attributes actually need to be vectorized
def consolidate_gauge(df):
    """ takes in gauge columns and normalizes them all to stiches per inch """
    try:
        df['gauge_per_inch'] = df.loc[:,'gauge']/df.loc[:,'gauge_divisor']
        df.drop(columns = ['gauge', 'gauge_divisor'], axis = 1, inplace = True)
    except:
        print("Error occured when consolidating gauge")
    return df

def encode_yarn_weights(df):
    """ uses yarn_weight_description to convert weights correspnding actual relative thicknesses
    indstry standards splits into 7 categories, but this goes more ganually """
    yarn_weights = {'Lace' : 1,
                    'Thread':1,
                    'Cobweb':1,
                    'Light Fingering':1.5,
                    'Fingering (14 wpi)': 2,
                    'Sport (12 wpi)': 3,
                    'DK / Sport' : 4,
                    'DK (11 wpi)' : 5,
                    'Worsted (9 wpi)':6,
                    'Aran / Worsted': 7,
                    'Aran (8 wpi)': 8,
                    'Bulky (7 wpi)':9,
                    'Super Bulky (5-6 wpi)':10,
                    'Jumbo (0-4 wpi)':11,
                    'No weight specified':5,
                    'Any gauge - designed for any gauge':5}
    ##--------------need to get yarn weight for other 
    try:
        df.replace({'yarn_weight_description':yarn_weights}, inplace=True)
        df.rename(columns = {'yarn_weight_description':'yarn_weight_code'}, inplace=True)
    except:
        print("okay - check out yarn_weight_description, something went wrong with the encoding")
    return df

# # NOT USED - using average for now
# def fill_max_yardage(df):
#     """ if there the yardage column is filled, but max_yardage is empty, it's because there isn't a 
#     range of yardage (in the case of one-sized non-clothing items).  This function fills that column in the c
#     case we only want to use min or max yardage """
#     df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
#     return df

def use_avg_yardage(df):
    df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
    df['yardage_avg'] = (df['yardage'] + df['yardage_max'])/2
    df.drop(columns=['yardage_max','yardage'], inplace=True)
    return df

def split_date_released(df):
    """ takes in 'generally_available' column and splits out into month and year 
    (to take into account seasonal purchases, and yearly impacts)"""
    try:
        df['generally_available'] = pd.to_datetime(df['generally_available'], utc=True)
        df['month_avail'] = df.generally_available.dt.month
        df['year_avail'] = df.generally_available.dt.year
     
        df.drop(columns = ['generally_available'], axis = 1, inplace = True)
    except:
        "print - Error occured when trying to split dates"
    return df

def get_list_metric_needles(x):
    """Get the list of metric needle sizes for knitting (can have several needle sizes for various parts of the pattern 
    and crochet hooks for finishing.  Generally the largest needle is the 
    one used for the main body (or largest portion of the project) """
    n=[]
    for i in range(len(x)):
        if x[i]['knitting'] == True:
            n.append(x[i]['metric'])
    return n

def get_needle_size(df):
    try:
        df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
    except:
        pass
    
    df['pattern_needle_sizes'] = df['pattern_needle_sizes'].apply(lambda x: get_list_metric_needles(x))
    df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: max(x) if (len(x)>=1) else None)
    return df

# def calculate_popularity(df):
#     pass
# # get_weighted_rating

def parse_out_single_category(df):
    df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x))
    df['categories'] = df['categories'].apply(lambda x: x[0])
    return df

def vectorize_categories(df):
    pass
# count_vect = CountVectorizer()

def vectorize_pattern_attributes(df):
    corpus = df['attributes'].tolist()
    
def impute_missing_values_or_zeros(df):
    df['difficulty_average'].fillna(df['difficulty_average'].median(), inplace=True)
    df['pattern_type_clothing'].fillna(df['pattern_type_clothing'].median(), inplace=True)
    df['pattern_needle_sizes'].fillna(df['pattern_needle_sizes'].median(), inplace=True)
    df['free'].fillna(df['free'].median(), inplace=True)
    df['downloadable'].fillna(df['downloadable'].median(), inplace=True)
    df['yarn_weight_code'].fillna(df['yarn_weight_code'].median(), inplace=True)
    df['gauge_per_inch'].fillna(df['gauge_per_inch'].median(), inplace=True)
#        'month_avail', 'year_avail', 
    df['yardage_avg'].fillna(df['yardage_avg'].median(), inplace=True)
    return df


# def one_hot_encode_binary(df):
    
#     feature_to_encode = ['pattern_type_clothing','free','downloadable'] 
#     dummies = pd.get_dummies(df[[feature_to_encode]], drop_first=True)
#     res = pd.concat([df, dummies], axis=1)
#     res = res.drop([feature_to_encode], axis=1)
#     return(res)

def apply_weighting_values(df):
    pass

#     using count isnt meaningful here ----------
# def get_weighted_difficulty_average(df):
#     overal_difficulty_average = df.difficulty_average.mean()

# def convert_to_sentence(x):
#     # concatenate words with '-'
    
#     # covert from list to sentence
#     x = ' '.join(word for word in x)
#     x = x.replace('-','')
#     #consolidatae and remove a number of words! right now 245-----------------------------
#     return x

# df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: ast.literal_eval(x))
# df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: convert_to_sentence(x))
# corpus = df['pattern_attributes']
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(corpus)
# features = vectorizer.get_feature_names()


#--------------------------------------------
# # This one works lovely 
# def transform_features(data):
#     data['yardage_avg'] = use_avg_yardage(df)
#     encode_yarn_weights(df)
#     data['Total_Income_Log'] = np.log(data['ApplicantIncome'] + data['CoapplicantIncome'])
#     data['LoanAmt_Term_Ratio_Log']=  np.log(data['LoanAmount']/data['Loan_Amount_Term'])
#     data['LoanAmount_Log'] = np.log(data['LoanAmount'])
#     data.drop(labels=['ApplicantIncome','CoapplicantIncome', 'LoanAmount'], axis=1, inplace=True)
#     data.reset_index(inplace=True)
#     return data

# add_total_income_log_object = FunctionTransformer(replace_income_with_total_income_log)
# add_loanamt_term_ratio_log_object = FunctionTransformer(add_LoanAmt_Term_Ratio_Log)
# add_loanamount_log_object = FunctionTransformer(replace_loanamount_with_loanamount_log)

transform_features = FunctionTransformer(transform_features)

In [79]:
df = pd.read_csv('data/consolidated_patterns.csv', low_memory=False)
df = clean_out_bad_data(df)

# features  #245 features

### Column Transform try

In [69]:
column_trans = ColumnTransformer(
     [('one_hot_encode', OneHotEncoder(dtype='int'), ['pattern_type_clothing','downloadable','free','pattern_type_names', 'categories']),
#       ('attributes_bow', CountVectorizer(), ['attributes'])],
     ],remainder='passthrough')

# >>> column_trans.fit(X)
# ColumnTransformer(transformers=[('categories', OneHotEncoder(dtype='int'),
#                                  ['city']),
#                                 ('title_bow', CountVectorizer(), 'title')],
#                   verbose_feature_names_out=False)

# >>> column_trans.get_feature_names_out()
# array(['city_London', 'city_Paris', 'city_Sallisaw', 'bow', 'feast',
# 'grapes', 'his', 'how', 'last', 'learned', 'moveable', 'of', 'the',
#  'trick', 'watson', 'wrath'], ...)

# >>> column_trans.transform(X).toarray()
# array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#        [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0],
#        [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]]...)

In [71]:
# column_trans.fit(df)
# column_trans.get_feature_names_out()

### Pipeline

In [26]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self
    
# transformer_step = ColumnTransformer([
#         ('impute_mean', SimpleImputer(strategy='mean'), ['age'])
#     ], remainder='passthrough')

# pipe = Pipeline([
#     ('select', transformer_step)
# ])
    
# ('one-hot-encode', OneHotEncoder(sparse=False))

ohe = OneHotEncoder(sparse=False)
imp = SimpleImputer()
imp_constant = SimpleImputer(strategy='constant')
imp_median = SimpleImputer(strategy='median', add_indicator=True)

scaler = StandardScaler()

# column_transfomer = ColumnTransformer(
#     [('encoder', ohe, ['pattern_type_clothing','free','downloadable']),],
# # #     [('encoder', ohe, ['pattern_type_clothing','free','downloadable','pattern_type_names','categories']),
# #      ('imputer', imp, ['pattern_type_clothing','free','downloadable'])],
#     remainder='passthrough') #any columns not named, pass through

# categorical_preprocessing = Pipeline([('ohe', OneHotEncoder(sparse=False))])
# numerical_preprocessing = Pipeline([('imputation', SimpleImputer())])


preprocesing_pipeline = Pipeline([
#     ("parse_out_single_category", DataframeFunctionTransformer(parse_out_single_category)),
    ("drop_identifying_columns", DataframeFunctionTransformer(drop_identifying_columns)), # remove pattern_type names and catefries after 
    ("cosolidate_gauge", DataframeFunctionTransformer(consolidate_gauge)),
    ("encode_yarn_weights", DataframeFunctionTransformer(encode_yarn_weights)),
    ("get_needle_size", DataframeFunctionTransformer(get_needle_size)), 
    ("split_date_released", DataframeFunctionTransformer(split_date_released)),
    ("use_avg_yardage", DataframeFunctionTransformer(use_avg_yardage)),
    ("impute_missing_values_or_zeros", DataframeFunctionTransformer(impute_missing_values_or_zeros)),
#     ("one_hot_encode_binary", DataframeFunctionTransformer(one_hot_encode_binary)),
#     ("apply_weighting_values", DataframeFunctionTransformer(apply_weighting_values)),
    ("one_hot_encode_and_impute_missing", column_transfomer),
#     ("standard_scale", scaler),
   
    
])

# categories - pick off top 3 and weighted onehot encode
# one hot encode ['pattern_type_clothing','free','downloadable']
# attributs - weighted count vecorize

# scale

# convert year down to 20 off and if before 2000 make 0



In [80]:
features = preprocesing_pipeline.fit(df)

# X.columns

In [83]:
features.get_feature_names()

AttributeError: 'Pipeline' object has no attribute 'get_feature_names'

In [82]:
the_features = preprocesing_pipeline.transform(df)

KeyError: "['pattern_id' 'name' 'name_permalink' 'photos_url' 'pattern_attributes'\n 'pattern_type_names' 'categories' 'rating_count' 'favorites_count'\n 'projects_count' 'rating_average' 'difficulty_count'\n 'queued_projects_count'] not found in axis"

In [81]:
transformed_df = pd.DataFrame(the_features, columns=features.get_feature_names())
transformed_df.head()


NameError: name 'the_features' is not defined

In [29]:
df1 = pd.DataFrame(X)
df1

# NEED SCALING!!
# NEED NAMES

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,1.0,0.0,1.0,1.816424,5.00,8.0,5.00,12.0,2006.0,93.0
1,0.0,1.0,0.0,1.0,0.0,1.0,2.435581,2.50,2.0,8.00,4.0,2006.0,370.0
2,0.0,1.0,1.0,0.0,0.0,1.0,2.851249,4.00,3.0,5.00,5.0,2007.0,437.0
3,0.0,1.0,0.0,1.0,0.0,1.0,1.865684,3.50,5.0,5.50,3.0,2006.0,234.0
4,0.0,1.0,1.0,0.0,1.0,0.0,2.853333,2.50,2.0,7.50,10.0,2007.0,460.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132838,1.0,0.0,0.0,1.0,0.0,1.0,1.571429,4.50,6.0,5.00,4.0,2013.0,110.0
132839,0.0,1.0,1.0,0.0,1.0,0.0,1.666667,3.75,2.0,4.00,12.0,2020.0,960.0
132840,0.0,1.0,0.0,1.0,0.0,1.0,1.200000,5.00,6.0,5.50,12.0,2020.0,836.0
132841,0.0,1.0,0.0,1.0,0.0,1.0,3.000000,4.50,3.0,5.25,1.0,2021.0,1730.0


In [87]:
# def get_needle_size(df):
#     try:
#         df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
#     except:
#         pass
#     df['pattern_needle_sizes'] = df['pattern_needle_sizes'].apply(lambda x: get_list_metric_needles(x))
#     df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: max(x) if (len(x)>=1) else None)
#     return df

# get_needle_size(df.iloc[0])

AttributeError: 'str' object has no attribute 'apply'

In [97]:
type(df['pattern_needle_sizes'][0])

str

In [79]:
df.reset_index(inplace=True, drop =True)

In [106]:
df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))

In [111]:
df['pattern_needle_sizes'][0][0]['knitting']

True

In [31]:
df1[2]

0         0.0
1         0.0
2         1.0
3         0.0
4         1.0
         ... 
132838    0.0
132839    1.0
132840    0.0
132841    0.0
132842    1.0
Name: 2, Length: 132843, dtype: float64

In [42]:
X.shape

(132843, 13)

In [45]:
from sklearn.metrics.pairwise import euclidean_distances
# From lecture
pattern_to_use = X[2].reshape(1,-1)
pattern_to_use
# pattern_to_use = preprocesing_pipeline.transform(pattern_to_use)

# # Find distances to other patterns
distances = euclidean_distances(X, pattern_to_use)
distances = distances.reshape(1,-1) 

# # Find the 3 indices with the minimum distance (highest similarity) to the car we're looking at
ordered_indices = distances.argsort()
closest_indices = ordered_indices[:3]

In [46]:
closest_indices

array([[     2,  73807,  27169, ...,  38168, 130250,  12837]])

In [ ]:
def get_needle_size(df):
    try:
        df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
    except:
        pass
    sizes =[]
    for i in range(len(df['pattern_needle_sizes']):
        if df['pattern_needle_sizes'][i][0]['knitting']=='True':
            sizes.append(df['pattern_needle_sizes'][i][0]['metric']
        else
            sizes.append('None')
                   
    return df

In [8]:
df.reset_index(drop=True, inplace=True)

In [14]:
try:
    df['pattern_needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
except:
    pass
sizes =[]
for i in df['pattern_needle_sizes']:
#     print(df['pattern_needle_sizes'][i])
    if i[0]['knitting']=='True':

        sizes.append(df['pattern_needle_sizes'][i][0]['metric'])
    else:
        sizes.append('None')

IndexError: list index out of range

In [ ]:
# # Distance from all other cars
# distances = euclidean_distances(X, looking_at_car)
# distances = distances.reshape(-1)   # Before it was (n_cars, 1)

# # Find the 3 indices with the minimum distance (highest similarity) to the car we're looking at
# ordered_indices = distances.argsort()
# closest_indices = ordered_indices[:3]

# # Get the cars for these indices
# closest_cars = cars.iloc[closest_indices]
# closest_cars

In [74]:
df.iloc[0]

pattern_id                                                                88
name                                                             Calorimetry
name_permalink                                                   calorimetry
favorites_count                                                        29838
projects_count                                                         19426
difficulty_average                                                   1.81642
difficulty_count                                                        7294
rating_average                                                       4.22529
queued_projects_count                                                   7895
rating_count                                                            6618
pattern_type_names                                                       hat
pattern_type_clothing                                                   True
photos_url                 https://images4-f.ravelrycache.com/uploads/cas...

In [42]:
df.pattern_type_names.unique()

array(['hat', 'socks', 'gloves', 'bag', 'baby', 'blanket', 'cardigan',
       'child', 'other', 'shawl', 'pullover', 'scarf', 'tee', 'toys',
       'vest', 'camisole', 'home', 'dishcloth', 'pet', 'jacket', 'shrug',
       'dress-suit', 'skirt', 'naughty', nan], dtype=object)

In [189]:
df.categories.unique() #193

array(['headband', 'mid-calf', 'beanie-toque', 'cardigan', 'babyblanket',
       'fingerless', 'slippers', 'shawl-wrap', 'pullover', 'scarf',
       'poncho', 'tee', 'onesies', 'ankle-sock', 'tube', 'other-socks',
       'beret-tam', 'animal', 'coat', 'vest', 'sleeveless-top', 'mittens',
       'earwarmers', 'cowl', 'market', 'convertible', 'balaclava',
       'dress', 'doll', 'bib', 'earflap', 'other-blanket',
       'christmasstocking', 'gloves', 'bedding', 'booties', 'bonnet',
       'pet-clothing', 'afghanblock', 'throw', 'costume', 'towel',
       'stocking', 'hangingornament', 'fashion-doll', 'scrubber', 'socks',
       'washcloth', 'other-hands', 'other-neck-torso', 'pixie', 'shorts',
       'hairaccessories', 'shrug', 'containers', 'drawstring', 'pillow',
       'other-decorative', 'brimmed', 'legwarmers', 'other-cozy',
       'other-accessories', 'tote', 'other-hat', 'knee-highs',
       'other-dollclothes', 'potholder', 'leggings', 'toeless', 'cup-mug',
       'electronics', 

In [212]:
def create_fabric_density_ratio(df):
    df['fabric_density'] = df['yarn_weight_code']/df['pattern_needle_sizes']
    return(df)
#     ratio of need to needle size and yarn_weight
df =create_fabric_density_ratio(df)


In [191]:
X.T

,0,1,2,3,4,5,8,9,10,11,...,188099,188110,188114,188117,188127,188148,188152,188153,188154,188156
favorites_count,29838,17795,3483,22539,1190,36858,29318,19644,47871,11364,...,709,576,88,224,598,151,103,353,647,357
projects_count,19426,10987,6019,7325,2469,9212,5281,6391,11192,2559,...,13,11,15,14,57,15,58,24,53,13
difficulty_average,1.81642,2.43558,2.85125,1.86568,2.85333,1.85447,2.26698,1.61973,2.48483,1.45052,...,2.75,5,2.33333,2.42857,6.5,1.57143,1.66667,1.2,3,1.33333
difficulty_count,7294,2670,921,2442,300,3209,1678,1825,3560,677,...,4,3,3,7,12,7,3,5,4,3
rating_average,4.22529,4.45524,4.4933,4.26759,4.22378,4.47137,4.42963,4.39892,4.44425,4.48071,...,5,5,5,4.71429,4.72727,4.5,5,5,4.75,4.66667
queued_projects_count,7895,3519,463,4534,219,7641,4458,3378,8203,1331,...,74,40,3,23,76,23,17,49,91,44
rating_count,6618,2480,896,2246,286,2951,1620,1662,3381,622,...,4,3,3,7,11,8,3,6,4,3
pattern_type_names,hat,socks,socks,hat,socks,hat,baby,hat,baby,blanket,...,hat,shawl,baby,hat,gloves,dishcloth,shawl,scarf,scarf,shawl
pattern_type_clothing,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,False,True,True,True,True
pattern_needle_sizes,5,2.5,NaN,3.5,2.5,4.5,3.5,4,5,4.5,...,4.5,3.75,3.75,3,1.5,4.5,3.75,5,4.5,3.5


In [ ]:

def create_average_rating(df):
    pass

### Visualize Pipeline

In [36]:
from sklearn import set_config
set_config(display='diagram')

pipeline

Pipeline(steps=[('cosolidate_gauge',
                 <__main__.DataframeFunctionTransformer object at 0x7ffbc0207e10>)])

In [ ]:
process difficulty and rating average down to 3 decimal places

#### columns I'm unsure about:
Downloadable - do I keep?
Make sure yarn weight is filled - which one?? and related it back to yarn weight name
** fix pattern needles size
row gauge?? is this needed?? probably not, other gauge should be fine
** need a function to impute missing values!! figure out what should be for each row
** if lots on a particular row, drop


In [ ]:
# 

#### Get pattern page link

In [ ]:
link = f'https://www.ravelry.com/patterns/library/{permalink}'